In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_64x64_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Gauss_64_Depth5/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.15598707983228896
Average test loss: 0.011658013568984138
Epoch 2/300
Average training loss: 0.0644736950662401
Average test loss: 0.009328207202669647
Epoch 3/300
Average training loss: 0.05750394963224729
Average test loss: 0.009705708081523578
Epoch 4/300
Average training loss: 0.05423388868570328
Average test loss: 0.008976573336041635
Epoch 5/300
Average training loss: 0.052451778199937606
Average test loss: 0.00797944614249799
Epoch 6/300
Average training loss: 0.04938272934820917
Average test loss: 0.008738897619148095
Epoch 7/300
Average training loss: 0.047450244002872045
Average test loss: 0.007640497989952564
Epoch 8/300
Average training loss: 0.046034170309702555
Average test loss: 0.008080495287560755
Epoch 9/300
Average training loss: 0.04472726974884669
Average test loss: 0.007413036768221193
Epoch 10/300
Average training loss: 0.04393243000242445
Average test loss: 0.0071025846277674035
Epoch 

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 22.17
Average PSNR for Projection Layer 1 across 2500 images: 24.47
Average PSNR for Projection Layer 2 across 2500 images: 25.55
Average PSNR for Projection Layer 3 across 2500 images: 26.33
Average PSNR for Projection Layer 4 across 2500 images: 26.79
Average PSNR for Projection Layer 0 across 2500 images: 23.44
Average PSNR for Projection Layer 1 across 2500 images: 26.10
Average PSNR for Projection Layer 2 across 2500 images: 27.80
Average PSNR for Projection Layer 3 across 2500 images: 28.68
Average PSNR for Projection Layer 4 across 2500 images: 29.51
Average PSNR for Projection Layer 0 across 2500 images: 24.04
Average PSNR for Projection Layer 1 across 2500 images: 27.49
Average PSNR for Projection Layer 2 across 2500 images: 29.54
Average PSNR for Projection Layer 3 across 2500 images: 30.68
Average PSNR for Projection Layer 4 across 2500 images: 31.48
Average PSNR for Projection Layer 0 across 2500 images: 24.56
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Gauss_64_Depth5/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 1.985914975060357
Average test loss: 0.011495959897008207
Epoch 2/300
Average training loss: 0.7068398025830587
Average test loss: 0.009443187534809112
Epoch 3/300
Average training loss: 0.4725997794204288
Average test loss: 0.008195544961425993
Epoch 4/300
Average training loss: 0.3704102176825205
Average test loss: 0.00795029717311263
Epoch 5/300
Average training loss: 0.31015565186076693
Average test loss: 0.007453564032912255
Epoch 6/300
Average training loss: 0.27099211049079897
Average test loss: 0.007117503167440494
Epoch 7/300
Average training loss: 0.24187704363134171
Average test loss: 0.00850981379341748
Epoch 8/300
Average training loss: 0.2181187744140625
Average test loss: 0.006951010062048833
Epoch 9/300
Average training loss: 0.2011922672457165
Average test loss: 0.007823510912143523
Epoch 10/300
Average training loss: 0.18734425718254513
Average test loss: 0.006395382797138558
Epoch 11/300
Aver

In [10]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 21.62
Average PSNR for Projection Layer 1 across 2500 images: 23.64
Average PSNR for Projection Layer 2 across 2500 images: 24.65
Average PSNR for Projection Layer 3 across 2500 images: 25.41
Average PSNR for Projection Layer 4 across 2500 images: 25.87
Average PSNR for Projection Layer 5 across 2500 images: 26.23
Average PSNR for Projection Layer 6 across 2500 images: 26.10
Average PSNR for Projection Layer 7 across 2500 images: 26.47
Average PSNR for Projection Layer 8 across 2500 images: 26.78
Average PSNR for Projection Layer 9 across 2500 images: 26.89
Average PSNR for Projection Layer 10 across 2500 images: 27.20
Average PSNR for Projection Layer 11 across 2500 images: 27.38
Average PSNR for Projection Layer 12 across 2500 images: 27.34
Average PSNR for Projection Layer 13 across 2500 images: 27.54
Average PSNR for Projection Layer 14 across 2500 images: 27.65
Average PSNR for Projection Layer 0 across 2500 images: 21.41
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Gauss_64_Depth5/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 15.894894397735596
Average test loss: 0.01205452611297369
Epoch 2/300
Average training loss: 6.3320272551642525
Average test loss: 0.025632556497222847
Epoch 3/300
Average training loss: 5.607419024573432
Average test loss: 0.008324858326878813
Epoch 4/300
Average training loss: 4.179052116394043
Average test loss: 0.010815686288807127
Epoch 5/300
Average training loss: 3.0709195115831163
Average test loss: 0.008234375040564273
Epoch 6/300
Average training loss: 2.522151819652981
Average test loss: 0.008259956325921747
Epoch 7/300
Average training loss: 2.02619872601827
Average test loss: 0.008655744524465667
Epoch 8/300
Average training loss: 1.6659804095162285
Average test loss: 0.006770922361355689
Epoch 9/300
Average training loss: 1.3956462364196778
Average test loss: 0.00844115963164303
Epoch 10/300
Average training loss: 1.144948079003228
Average test loss: 0.0066401605581243835
Epoch 11/300
Average trai

In [12]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 21.42
Average PSNR for Projection Layer 1 across 2500 images: 23.46
Average PSNR for Projection Layer 2 across 2500 images: 24.36
Average PSNR for Projection Layer 3 across 2500 images: 25.11
Average PSNR for Projection Layer 4 across 2500 images: 25.33
Average PSNR for Projection Layer 5 across 2500 images: 25.78
Average PSNR for Projection Layer 6 across 2500 images: 26.18
Average PSNR for Projection Layer 7 across 2500 images: 26.18
Average PSNR for Projection Layer 8 across 2500 images: 26.58
Average PSNR for Projection Layer 9 across 2500 images: 26.48
Average PSNR for Projection Layer 10 across 2500 images: 26.81
Average PSNR for Projection Layer 11 across 2500 images: 26.78
Average PSNR for Projection Layer 12 across 2500 images: 27.02
Average PSNR for Projection Layer 13 across 2500 images: 26.92
Average PSNR for Projection Layer 14 across 2500 images: 27.27
Average PSNR for Projection Layer 15 across 2500 images: 27.38
Av